# Fe55クラスターにC atomを供給する計算@1500K

MDの参考：https://docs.matlantis.com/atomistic-simulation-tutorial/ja/6_2_md-nvt.html

In [9]:
from fairchem.core import OCPCalculator

import ase
from ase import Atoms
from ase.io import read, Trajectory
import numpy as np
from ase.optimize import LBFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.langevin import Langevin
from ase.md import MDLogger
from ase import units
from time import perf_counter
import os

#　クラスター初期構造の読み込み
atoms =  ase.io.read("../structure_build/Fe55.xyz")

# 学習済みモデル(NNP)の読み込み
calculator = OCPCalculator(
    model_name="EquiformerV2-31M-S2EF-OC20-All+MD", # 申請が通ったらOMat24のモデルに変更
    local_cache="pretrained_models",
    cpu=True,
)

atoms.calc = calculator

# 構造緩和
opt = LBFGS(atoms)
opt.run(fmax=0.05)

# MD計算の条件
time_step    = 1.0    # fsec
temperature  = 1500   # Kelvin
num_md_steps = 100     # Total MD step, for testing.
# num_md_steps = 1e6    # Total MD step, for actual run.
num_interval = 1      # Output printing interval
friction_coeff = 0.005
threshold = 2.3          # Å (Fe–C bond cutoff)
threshold_cc = 1.5       # Å (C–C bond cutoff)
target_gas = 8           # Desired number of gas-phase C atoms
cell_size = 20.0         # Å


# 出力設定
output_dir = "output"
# ディレクトリが存在しない場合のみ作成
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_filename = f"./output/cntgrowth_{atoms.symbols}_{temperature}K"

print("output_filename = ",output_filename)
log_filename = output_filename + ".log"
print("log_filename = ",log_filename)
traj_filename = output_filename + ".traj"
print("traj_filename = ",traj_filename)

# 初速の設定
MaxwellBoltzmannDistribution(atoms, temperature_K=temperature,force_temp=True)
Stationary(atoms)  # 運動量の総和をゼロベクトルにする

# C原子数制御
fe_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Fe']

def maintain_gas_atoms():
    """Observer function to keep exactly `target_gas` unbound C atoms in the cell."""
    # ステップ数が 0 のときはスキップ
    if dyn.get_number_of_steps() == 0:
        return
    
    global atoms, fe_indices, threshold, threshold_cc
    pos = atoms.get_positions()
    fe_pos = pos[fe_indices]
    #  Current C atom indices
    c_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'C']
    
    # Identify gas-phase C atoms (no Fe–C within threshold)
    gas_indices = [
        i for i in c_indices
        if (np.linalg.norm(pos[i] - fe_pos, axis=1) > threshold).all()
    ]

    # Number to add
    n_add = target_gas - len(gas_indices)
    new_indices = []

    for _ in range(max(n_add, 0)):
        while True:
            pos_new = np.random.rand(3) * cell_size
            # Check Fe–C overlap
            if (np.linalg.norm(pos_new - fe_pos, axis=1) < threshold).any():
                continue
            # Check C–C overlap
            if c_indices:
                c_pos = pos[c_indices]
                if (np.linalg.norm(pos_new - c_pos, axis=1) < threshold_cc).any():
                    continue
            break
        # Add new C atom
        atoms += Atoms('C', positions=[pos_new])
        new_idx = len(atoms) - 1
        new_indices.append(new_idx)

    # Assign Maxwell–Boltzmann velocities *only* to new atoms
    if new_indices:
        new_atoms = atoms[new_indices]
        MaxwellBoltzmannDistribution(new_atoms, temperature_K=temperature)
        # Copy velocities back
        v_all = atoms.get_velocities()
        v_new = new_atoms.get_velocities()
        for idx, vel in zip(new_indices, v_new):
            v_all[idx] = vel
        atoms.set_velocities(v_all)

# Print statements
def print_dyn():
    imd = dyn.get_number_of_steps()
    etot  = atoms.get_total_energy()
    temp_K = atoms.get_temperature()
    elapsed_time = perf_counter() - start_time
    print(f"  {imd: >3}   {etot:.3f}    {temp_K:.2f}   {elapsed_time:.3f}")

# ループの前に一度だけLoggerを用意する
traj = Trajectory(traj_filename, 'w')  # 初回はヘッダー付で新規作成
logger = MDLogger(
    None,           # integrator は後で attach 時に渡す
    atoms,
    log_filename,
    mode='w',       # 書き込みモード
    header=True,    # ヘッダーは１回だけ
    stress=False,
    peratom=True
)

# run MD
for step in range(1, num_md_steps + 1):
    dyn = Langevin(atoms, time_step*units.fs, friction=friction_coeff, temperature_K=temperature, trajectory=None, logfile=None)
    dyn.attach(maintain_gas_atoms, interval=1)
    dyn.attach(print_dyn, interval=1)
    dyn.attach(logger, interval=1)

    dyn.run(1)

    # トラジェクトリは自分で追記
    traj.write(atoms)

    # ２回目以降は logger を append モードに変更
    if step == 1:
        logger.mode = 'a'
        logger.header = False

INFO:root:Checking local cache: pretrained_models for model EquiformerV2-31M-S2EF-OC20-All+MD
/home/oxygen/miniforge3/envs/fair-chem-cpu/lib/python3.12/site-packages/fairchem/core/common/relaxation/ase_utils.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded fil

       Step     Time          Energy          fmax
LBFGS:    0 21:32:39      106.226654       10.241766
LBFGS:    1 21:32:39       52.717384        6.003488
LBFGS:    2 21:32:40        5.957635        2.339214
LBFGS:    3 21:32:41       -0.359707        0.943126
LBFGS:    4 21:32:42       -0.744448        0.264602
LBFGS:    5 21:32:42       -0.778530        0.163984
LBFGS:    6 21:32:43       -0.801449        0.029290
output_filename =  ./output/cntgrowth_Fe55_1500K
log_filename =  ./output/cntgrowth_Fe55_1500K.log
traj_filename =  ./output/cntgrowth_Fe55_1500K.traj
    0   9.862    1500.00   671.685
    1   66.362    1500.66   673.299
    0   66.362    1500.66   673.301
    1   66.124    1495.05   674.250
    0   66.124    1495.05   674.252
    1   65.918    1498.69   675.145
    0   65.918    1498.69   675.148
    1   65.918    1498.27   676.033
    0   65.918    1498.27   676.035
    1   66.013    1496.34   676.907
    0   66.013    1496.34   676.908
    1   65.981    1484.11   677.

In [10]:
from ase.io import Trajectory
from ase.io.trajectory import TrajectoryReader
import sys

path = "../visualization"
sys.path.append(os.path.abspath(path))
from ase_nglview import view_ngl

traj = TrajectoryReader(traj_filename)
view_ngl(traj)

NGLWidget(max_frame=99)